<a href="https://colab.research.google.com/github/mnassar/segfault/blob/main/SegFault_TABLE_VI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install foolbox

     |████████████████████████████████| 1.7MB 5.9MB/s 
     |████████████████████████████████| 61kB 9.9MB/s 
     |████████████████████████████████| 174kB 51.8MB/s 
     |████████████████████████████████| 71kB 10.7MB/s 
ERROR: google-colab 1.0.0 has requirement requests~=2.23.0, but you'll have requests 2.25.1 which is incompatible.
ERROR: datascience 0.10.6 has requirement folium==0.2.1, but you'll have folium 0.8.3 which is incompatible.
  Found existing installation: requests 2.23.0
    Uninstalling requests-2.23.0:
      Successfully uninstalled requests-2.23.0


In [2]:
import tensorflow.compat.v2 as tf 
tf.enable_v2_behavior()
from sklearn.model_selection import train_test_split

from xgboost import XGBClassifier
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score
from skimage.segmentation import  felzenszwalb, slic, quickshift
from skimage.segmentation import mark_boundaries
from skimage.util import img_as_float
import numpy as np
import foolbox as fb
import matplotlib.pyplot as plt

In [3]:
!wget --no-check-certificate 'https://docs.google.com/uc?export=download&id=1H4KEE0Vp8DFZOe_QfcxqOxEVnpun-uka' -O CIFAR10model.h5

--2021-06-13 08:46:28--  https://docs.google.com/uc?export=download&id=1H4KEE0Vp8DFZOe_QfcxqOxEVnpun-uka
Resolving docs.google.com (docs.google.com)... 74.125.203.100, 74.125.203.102, 74.125.203.138, ...
Connecting to docs.google.com (docs.google.com)|74.125.203.100|:443... connected.
HTTP request sent, awaiting response... 302 Moved Temporarily
Location: https://doc-0s-68-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/msbjnq4jaidovt66h5krn8mmk54tnviu/1623573975000/13345485370729250143/*/1H4KEE0Vp8DFZOe_QfcxqOxEVnpun-uka?e=download [following]
--2021-06-13 08:46:28--  https://doc-0s-68-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/msbjnq4jaidovt66h5krn8mmk54tnviu/1623573975000/13345485370729250143/*/1H4KEE0Vp8DFZOe_QfcxqOxEVnpun-uka?e=download
Resolving doc-0s-68-docs.googleusercontent.com (doc-0s-68-docs.googleusercontent.com)... 74.125.203.132, 2404:6800:4008:c03::84
Connecting to doc-0s-68-docs.googleusercontent.com (doc-0s-68-d

In [4]:
# load the cifar classifier
from tensorflow.keras.models import load_model

pretrained_model = load_model('CIFAR10model.h5')
pretrained_model.trainable = False
pretrained_model.summary()


Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_6 (Conv2D)            (None, 32, 32, 32)        896       
_________________________________________________________________
activation_6 (Activation)    (None, 32, 32, 32)        0         
_________________________________________________________________
batch_normalization_6 (Batch (None, 32, 32, 32)        128       
_________________________________________________________________
conv2d_7 (Conv2D)            (None, 32, 32, 32)        9248      
_________________________________________________________________
activation_7 (Activation)    (None, 32, 32, 32)        0         
_________________________________________________________________
batch_normalization_7 (Batch (None, 32, 32, 32)        128       
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 16, 16, 32)       

In [5]:
import tensorflow_datasets as tfds

(ds_train, ds_test), ds_info = tfds.load(
    'cifar10',
    split=['train', 'test'],
    shuffle_files=True,
    as_supervised=True,
    with_info=True,
)

print (ds_info)


Shuffling and writing examples to /root/tensorflow_datasets/cifar10/3.0.2.incomplete7RKTAC/cifar10-train.tfrecord


Shuffling and writing examples to /root/tensorflow_datasets/cifar10/3.0.2.incomplete7RKTAC/cifar10-test.tfrecord


Dataset cifar10 downloaded and prepared to /root/tensorflow_datasets/cifar10/3.0.2. Subsequent calls will reuse this data.
tfds.core.DatasetInfo(
    name='cifar10',
    version=3.0.2,
    description='The CIFAR-10 dataset consists of 60000 32x32 colour images in 10 classes, with 6000 images per class. There are 50000 training images and 10000 test images.',
    homepage='https://www.cs.toronto.edu/~kriz/cifar.html',
    features=FeaturesDict({
        'id': Text(shape=(), dtype=tf.string),
        'image': Image(shape=(32, 32, 3), dtype=tf.uint8),
        'label': ClassLabel(shape=(), dtype=tf.int64, num_classes=10),
    }),
    total_num_examples=60000,
    splits={
        'test': 10000,
        'train': 50000,
    },
    supervised_keys=('image', 'label'),
    citation="""@TECHREPORT{Krizhevsky09learningmultiple,
        author = {Alex Krizhevsky},
        title = {Learning multiple layers of features from tiny images},
        institution = {},
        year = {2009}
    }""",
    

In [6]:
# normalize images 

# these are the numbers used during training the model 
mean = 120.70748
std = 64.150024
bound_min = (0-mean)/std
bound_max = (255-mean)/std
BATCH_SIZE=128

def normalize_img(image, label):
  """Normalizes images: `uint8` -> `float32`."""
  # return tf.cast(image, tf.float32) / 255., tf.one_hot(label, 10)
  return (tf.cast(image, tf.float32) - mean) / std, tf.one_hot(label, 10)


ds_train = ds_train.map(
    normalize_img, num_parallel_calls=tf.data.experimental.AUTOTUNE)
ds_train = ds_train.cache()
ds_train = ds_train.shuffle(ds_info.splits['train'].num_examples)
ds_train = ds_train.batch(BATCH_SIZE)
ds_train = ds_train.prefetch(tf.data.experimental.AUTOTUNE)

ds_test = ds_test.map(
    normalize_img, num_parallel_calls=tf.data.experimental.AUTOTUNE)
ds_test = ds_test.batch(BATCH_SIZE)
ds_test = ds_test.cache()
ds_test = ds_test.prefetch(tf.data.experimental.AUTOTUNE)

# Dataset Preparation

In [7]:
NB_BATCHES = 30 # means that we will have NB_BATCHES radnom normal batches
BATCH_SIZE=128
# and NB_BATCHES random adversarial batches each coming from a different normal batch and a different epsilon 

Choose Attack

In [8]:
fmodel = fb.models.TensorFlowModel(model=pretrained_model, bounds=(bound_min, bound_max))
#attack = fb.attacks.PGD()
# attack = fb.attacks.L2CarliniWagnerAttack(steps=500)
#attack = fb.attacks.FGSM()
attack = fb.attacks.LinfDeepFoolAttack()

Instructions for updating:
Use `tf.config.list_physical_devices('GPU')` instead.


Instructions for updating:
Use `tf.config.list_physical_devices('GPU')` instead.


In [9]:
ds_experiment = []
ds_experiment_f = []
preds = [] 
fpreds = [] 
epsilons = [0.02, 0.06, 0.1]

fmodel = fb.models.TensorFlowModel(model=pretrained_model, bounds=(bound_min, bound_max))
gen = iter(ds_train)

for b in range(NB_BATCHES): 
  images, labels = gen.next()
  preds.append(pretrained_model.predict(images))
  ds_experiment.append(images) 
  labels_class = tf.argmax(labels, axis=1)
  # raw, fimages, is_adv = attack(fmodel, images, criterion=fb.criteria.Misclassification(labels_class), epsilons=None) # free epsilon for C&W 
  raw, fimages, is_adv = attack(fmodel, images, criterion=fb.criteria.Misclassification(labels_class),epsilons=epsilons[b%3])
  ds_experiment_f.append(fimages)
  fpreds.append(pretrained_model.predict(fimages))
  

In [10]:
ds_experiment += ds_experiment_f
len(ds_experiment)

60

In [11]:
from keras import backend as K
# pretrained_model.summary()
NB_LAYERS=20
NB_NODES_PER_LAYER=200

In [12]:
inp = pretrained_model.input  
layers_= pretrained_model.layers[-NB_LAYERS:]
outputs = [lay.output for lay in layers_]     
intermediate_model = K.function([inp], outputs) 


# select NB_NODES_PER_LAYER random nodes from each selected layer
print ("these nodes will be used to compute the IQR-"+str(NB_LAYERS*NB_NODES_PER_LAYER))

node_indices=[]
for lay in layers_[:-1]:
  # we omit the first dim (batch dim) of each layer 
  node_indices.append([[np.random.randint(0,d) for d in lay.output.shape[1:]] for s in range(NB_NODES_PER_LAYER)])
  # print("%s:" % lay.name)
  

# add the last layer 
node_indices.append([[x] for x in range(10)])
# print("%s:" % layers_[-1].name)
print(intermediate_model)

these nodes will be used to compute the IQR-4000
<function function.<locals>.func at 0x7fea7b345680>


# IQR calculations with segmentation

In [13]:
from skimage.segmentation import  felzenszwalb, slic, quickshift
from skimage.segmentation import mark_boundaries
from skimage.util import img_as_float



In [53]:
# SLIC experiment 
# number of segments 
# n = 32 
# n = 64 
# n = 128
# QuickShift experiment s-0,1,2 m=10,20
s = 0
m = 10
# felzenszwalb experiment s=1,10,100,1000
# s = 1000


In [54]:

%%time 
iqr_all = []
iqr_10_all = [] 
iqr_4k_all =[]
for b in range(2*NB_BATCHES):
  print("batch %d:" % b) 
  images = ds_experiment[b]
  preds = pretrained_model.predict(images)
  preds_value = tf.reduce_max(preds, axis=1)
  preds_idx = tf.argmax(preds, axis=1)
  preds_layers = intermediate_model(images)
  preds_4k = [] 
  for u in range(NB_LAYERS): # loop through the last 10 layers 
    for v in node_indices[u]: # loop through the 10 random nodes for that layer 
      t = tuple(v)
      # print((0,*t))
      preds_4k.append ( preds_layers[u][(...,*t)] )
  preds_4k = np.array(preds_4k).T
  # print (preds_value)
  # print (preds_idx)
  # print ( preds_value == tf.gather_nd(preds, list(zip(range(BATCH_SIZE), preds_idx))) )
  iqr = [] 
  iqr10 = [] 
  iqr4k = []
  
  # choose your seg! 
  # segImages = [ slic(img, n_segments=n) for img in images ]
  segImages = [ quickshift(img, sigma=s, max_dist=m) for img in images]
  # segImages = [ felzenszwalb(img, scale=s) for img in images]

  max_segs = [ segImg.max() for segImg in segImages ] 
  max_n_segs = max (max_segs)
  # max_n_segs = max (segImg.max() for segImg in segImages)
  print (max_n_segs)
  # input()
  # mask = np.ones((1,32,32,3))

  for i in range(max_n_segs+1):     # i is seg number here 
    mask = np.ones((BATCH_SIZE,32,32,3))
    # mask the segment for all images in the batch 
    for j in range(BATCH_SIZE): # j is image number here 
      # print (segImages[j]==i)
      # input()
      mask[j,(segImages[j]==i),:] = 0

    images_0 = images * mask 
    # for testing the mask 
    # choose a number (image in batch)
    # t = 11
    # print (segImages[t].max())
    # plt.figure(figsize = (1,1))
    # plt.axis('off')
    # plt.imshow((images[t].numpy() * std + mean).astype(np.uint8))
    # plt.show()
    # plt.figure(figsize = (1,1))
    # plt.axis('off')
    # plt.imshow((images_0[t].numpy() * std + mean).astype(np.uint8))
    # plt.show()
    # input()
    
    preds_0 = pretrained_model.predict(np.array(images_0))
    preds_value_0 = tf.gather_nd(preds_0, list(zip(range(BATCH_SIZE), preds_idx))) 
    iqr.append(abs(preds_value - preds_value_0)) 
    iqr10.append(abs(preds - preds_0))
    preds_layers_0 = intermediate_model(images_0)
    preds_4k_0 = [] 
    for u in range(NB_LAYERS): # loop through the last 10 layers ]
        for v in node_indices[u]: # loop through the 10 random nodes for that layer 
          t = tuple(v) 
          preds_4k_0.append ( preds_layers_0[u][(...,*t)] )
    preds_4k_0 = np.array(preds_4k_0).T
    iqr4k.append(abs(preds_4k - preds_4k_0))


  # for some images the number of segments is less than max_n_segs
  # this results in a mask of all ones, the same image will be predicted 
  # In this case the difference of predicted value(s) will be 0 
  # to ignore these 0s we transform them to NAN and we use NANpercentile function 
  # insted of percentile function 

  # replace iqr vals of 0 with nan 
  iqr = np.array(iqr)
  iqr10 = np.array(iqr10)
  iqr4k = np.array(iqr4k) 
  

  for j in range(BATCH_SIZE):
    iqr[max_segs[j]+1:,j] =  np.nan
    iqr10[max_segs[j]+1:,j,:] = np.nan
    iqr4k[max_segs[j]+1:,j,:] = np.nan
 
  # nanpercentile is a bit slow, can be enhanced 
  a,b = np.nanpercentile (iqr4k, [25, 75], axis=0) 
  iqr_4k_vals = b - a
  a,b = np.nanpercentile(iqr10, [25, 75], axis=0) 
  iqr_10_vals = b - a
  a,b = np.nanpercentile(iqr, [25, 75], axis=0) 
  iqr_vals = b - a
  
  iqr_all.append(iqr_vals)
  iqr_10_all.append(iqr_10_vals)
  iqr_4k_all.append(iqr_4k_vals)



batch 0:
664
batch 1:
499
batch 2:
493
batch 3:
455
batch 4:
468
batch 5:
623
batch 6:
523
batch 7:
594
batch 8:
565
batch 9:
624
batch 10:
641
batch 11:
454
batch 12:
515
batch 13:
620
batch 14:
470
batch 15:
516
batch 16:
633
batch 17:
532
batch 18:
417
batch 19:
567
batch 20:
561
batch 21:
634
batch 22:
455
batch 23:
544
batch 24:
506
batch 25:
450
batch 26:
584
batch 27:
533
batch 28:
595
batch 29:
573
batch 30:
685
batch 31:
650
batch 32:
687
batch 33:
461
batch 34:
584
batch 35:
752
batch 36:
529
batch 37:
650
batch 38:
713
batch 39:
629
batch 40:
727
batch 41:
628
batch 42:
524
batch 43:
706
batch 44:
686
batch 45:
536
batch 46:
689
batch 47:
705
batch 48:
419
batch 49:
654
batch 50:
712
batch 51:
645
batch 52:
590
batch 53:
717
batch 54:
538
batch 55:
618
batch 56:
723
batch 57:
519
batch 58:
608
batch 59:
810
CPU times: user 2h 9min, sys: 2min 21s, total: 2h 11min 22s
Wall time: 2h 10min 7s


# classification

#1_D

In [55]:
X = np.array(iqr_all).flatten()
y = np.concatenate( ( np.zeros(NB_BATCHES*BATCH_SIZE), np.ones(NB_BATCHES*BATCH_SIZE) ) )

score = cross_val_score(XGBClassifier(), X.reshape(-1,1), y, cv=2)
print (score)

[0.7359375  0.72473958]


In [56]:
import math
print (len([x for x in X if math.isnan(x)]))
print (len(X)) 

0
7680


In [57]:
X_train, X_test, y_train, y_test = train_test_split(X.reshape(-1,1), y, test_size=0.2)
# rdm = RandomForestClassifier().fit(X_train,y_train)
# svc = SVC(probability=True).fit(X_train,y_train)
xgb = XGBClassifier().fit(X_train,y_train)

In [58]:
# print(rdm.score(X_train,y_train))
# print(rdm.score(X_test,y_test))
# # print(svc.score(X_train,y_train))
# print(svc.score(X_test,y_test))
# print(xgb.score(X_train,y_train))
print(xgb.score(X_test,y_test))

0.7265625


In [59]:
# AUC 
from sklearn.metrics import roc_curve, roc_auc_score

# rdm_probs = rdm.predict_proba(X_test)[:,1]
# svc_probs = svc.predict_proba(X_test)[:,1]
xgb_probs = xgb.predict_proba(X_test)[:,1]

# rdm_auc = roc_auc_score(y_test, rdm_probs)
# svc_auc = roc_auc_score(y_test, svc_probs)
xgb_auc = roc_auc_score(y_test, xgb_probs)

# print('Random Forest: AUROC = %.3f' %(rdm_auc) )
# print('SVC: AUROC = %.3f' %(svc_auc) )
print('RaXGBClassifier: AUROC = %.3f' %(xgb_auc) )

RaXGBClassifier: AUROC = 0.776


# IQR-10D

In [60]:
X = np.array(iqr_10_all).reshape(-1,10) 
y = np.concatenate( ( np.zeros(NB_BATCHES*BATCH_SIZE), np.ones(NB_BATCHES*BATCH_SIZE) ) )

score = cross_val_score(XGBClassifier(), X, y, cv=2)
print (score)

[0.75625    0.74869792]


In [61]:
X_train, X_test, y_train, y_test = train_test_split( X, y, test_size=0.2)
# rdm = RandomForestClassifier().fit(X_train,y_train)
# svc = SVC(probability=True).fit(X_train,y_train)
xgb = XGBClassifier().fit(X_train,y_train)

# print(rdm.score(X_test,y_test))
# print(svc.score(X_test,y_test))
print(xgb.score(X_test,y_test))

0.7649739583333334


In [62]:
# AUC 
from sklearn.metrics import roc_curve, roc_auc_score

# rdm_probs = rdm.predict_proba(X_test)[:,1]
# svc_probs = svc.predict_proba(X_test)[:,1]
xgb_probs = xgb.predict_proba(X_test)[:,1]

# rdm_auc = roc_auc_score(y_test, rdm_probs)
# svc_auc = roc_auc_score(y_test, svc_probs)
xgb_auc = roc_auc_score(y_test, xgb_probs)

# print('Random Forest: AUROC = %.3f' %(rdm_auc) )
# print('SVC: AUROC = %.3f' %(svc_auc) )
print('RaXGBClassifier: AUROC = %.3f' %(xgb_auc) )

RaXGBClassifier: AUROC = 0.839


# IQR-4K-D


# Classification

In [63]:
# We try a very basic classification 
X = np.array(iqr_4k_all).reshape(-1,3810) 
y = np.concatenate( ( np.zeros(NB_BATCHES*BATCH_SIZE), np.ones(NB_BATCHES*BATCH_SIZE) ) )

score = cross_val_score(XGBClassifier(), X, y, cv=2)
print (score)

[0.77916667 0.77161458]


In [ ]:
X_train, X_test, y_train, y_test = train_test_split( X, y, test_size=0.2)
# rdm = RandomForestClassifier().fit(X_train,y_train)
# svc = SVC(probability=True).fit(X_train,y_train)
xgb = XGBClassifier().fit(X_train,y_train)

# print(rdm.score(X_test,y_test))
# print(svc.score(X_test,y_test))
print(xgb.score(X_test,y_test))

In [65]:
# AUC 
from sklearn.metrics import roc_curve, roc_auc_score

# rdm_probs = rdm.predict_proba(X_test)[:,1]
# svc_probs = svc.predict_proba(X_test)[:,1]
xgb_probs = xgb.predict_proba(X_test)[:,1]

# rdm_auc = roc_auc_score(y_test, rdm_probs)
# svc_auc = roc_auc_score(y_test, svc_probs)
xgb_auc = roc_auc_score(y_test, xgb_probs)

# print('Random Forest: AUROC = %.3f' %(rdm_auc) )
# print('SVC: AUROC = %.3f' %(svc_auc) )
print('RaXGBClassifier: AUROC = %.3f' %(xgb_auc) )

RaXGBClassifier: AUROC = 0.835
RaXGBClassifier: AUROC = 0.835
